In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [3]:
df = pd.read_csv("smart_home_energy_consumption_large.csv")   # use your actual file name if different
df.head()


,Home ID,Appliance Type,Energy Consumption (kWh),Time,Date,Outdoor Temperature (°C),Season,Household Size
0,94,Fridge,0.20,21:12,2023-12-02,-1.0,Fall,2
1,435,Oven,0.23,20:11,2023-08-06,31.1,Summer,5
2,466,Dishwasher,0.32,06:39,2023-11-21,21.3,Fall,3
3,496,Heater,3.92,21:56,2023-01-21,-4.2,Winter,1
4,137,Microwave,0.44,04:31,2023-08-26,34.5,Summer,5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Home ID                   100000 non-null  int64  
 1   Appliance Type            100000 non-null  object 
 2   Energy Consumption (kWh)  100000 non-null  float64
 3   Time                      100000 non-null  object 
 4   Date                      100000 non-null  object 
 5   Outdoor Temperature (°C)  100000 non-null  float64
 6   Season                    100000 non-null  object 
 7   Household Size            100000 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 6.1+ MB


In [5]:
df.isnull().sum()


Home ID                     0
Appliance Type              0
Energy Consumption (kWh)    0
Time                        0
Date                        0
Outdoor Temperature (°C)    0
Season                      0
Household Size              0
dtype: int64

In [7]:
hourly_energy = df.groupby('Time')['Energy Consumption (kWh)'].mean()
hourly_energy.head()

Time
00:00    1.568308
00:01    1.472121
00:02    1.552254
00:03    1.488214
00:04    1.441714
Name: Energy Consumption (kWh), dtype: float64

In [15]:
# Fill missing energy values with the mean
df = df.fillna({'Energy Consumption (kWh)': df['Energy Consumption (kWh)'].mean()})
# Drop rows if Timestamp is missing (very important)
df.dropna(subset=['Time'], inplace=True)

In [16]:
df.columns


Index(['Home ID', 'Appliance Type', 'Energy Consumption (kWh)', 'Time', 'Date',
       'Outdoor Temperature (°C)', 'Season', 'Household Size'],
      dtype='object')

In [17]:
# Combine Date and Time into one Timestamp column
df['Timestamp'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])

# Set Timestamp as index
df.set_index('Timestamp', inplace=True)

df.head()


,Home ID,Appliance Type,Energy Consumption (kWh),Time,Date,Outdoor Temperature (°C),Season,Household Size
Timestamp,,,,,,,,
2023-12-02 21:12:00,94,Fridge,0.20,21:12,2023-12-02,-1.0,Fall,2
2023-08-06 20:11:00,435,Oven,0.23,20:11,2023-08-06,31.1,Summer,5
2023-11-21 06:39:00,466,Dishwasher,0.32,06:39,2023-11-21,21.3,Fall,3
2023-01-21 21:56:00,496,Heater,3.92,21:56,2023-01-21,-4.2,Winter,1
2023-08-26 04:31:00,137,Microwave,0.44,04:31,2023-08-26,34.5,Summer,5


In [18]:
# Calculate IQR for Energy Consumption
Q1 = df['Energy Consumption (kWh)'].quantile(0.25)
Q3 = df['Energy Consumption (kWh)'].quantile(0.75)
IQR = Q3 - Q1

lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

# Remove outliers
df = df[(df['Energy Consumption (kWh)'] >= lower_limit) &
        (df['Energy Consumption (kWh)'] <= upper_limit)]

df.shape


(92003, 8)

In [20]:
# Hourly energy consumption
hourly_data = df.resample('h')['Energy Consumption (kWh)'].sum()

# Daily energy consumption
daily_data = df.resample('D')['Energy Consumption (kWh)'].sum()

hourly_data.head(), daily_data.head()


(Timestamp
 2023-01-01 00:00:00    11.48
 2023-01-01 01:00:00    17.92
 2023-01-01 02:00:00    26.27
 2023-01-01 03:00:00    12.16
 2023-01-01 04:00:00    15.61
 Freq: h, Name: Energy Consumption (kWh), dtype: float64,
 Timestamp
 2023-01-01    345.84
 2023-01-02    306.59
 2023-01-03    280.07
 2023-01-04    326.47
 2023-01-05    331.37
 Freq: D, Name: Energy Consumption (kWh), dtype: float64)

In [21]:
from sklearn.preprocessing import MinMaxScaler

# Convert hourly data to DataFrame
hourly_df = hourly_data.to_frame(name='Energy_kWh')

# Apply Min-Max Scaling
scaler = MinMaxScaler()
hourly_df['Energy_Scaled'] = scaler.fit_transform(hourly_df[['Energy_kWh']])

hourly_df.head()


,Energy_kWh,Energy_Scaled
Timestamp,,
2023-01-01 00:00:00,11.48,0.297095
2023-01-01 01:00:00,17.92,0.475242
2023-01-01 02:00:00,26.27,0.706224
2023-01-01 03:00:00,12.16,0.315906
2023-01-01 04:00:00,15.61,0.411342


In [22]:
from sklearn.model_selection import train_test_split

# Features (X) and Target (y)
X = hourly_df[['Energy_Scaled']]
y = hourly_df['Energy_Scaled']

# First split: Train (70%) and Temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, shuffle=False
)

# Second split: Validation (15%) and Test (15%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, shuffle=False
)

# Check sizes
print("Train size:", X_train.shape)
print("Validation size:", X_val.shape)
print("Test size:", X_test.shape)


Train size: (6148, 1)
Validation size: (1318, 1)
Test size: (1318, 1)


In [23]:
hourly_df.to_csv("hourly_cleaned_energy.csv")
